# **Quantitative Value Strategy**

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf

In [2]:
# Importing list of stocks 
stock_names = pd.read_csv('sp_500_stocks.csv')

In [8]:
# Create a new dataFrame with specific information for the above stocks 
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
data_list = []

# Fetch and process each stock
for ticker in stock_names['Ticker']:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    try:
        price = info.get('currentPrice', None) 
        pe_ratio = info.get('trailingPE', None) 
    except KeyError: 
        price = None
        pe_ratio = None

    new_row = {
        'Ticker': ticker,
        'Price': price,
        'Price-to-Earnings Ratio': pe_ratio,
        'Number of Shares to Buy': None 
    }
    data_list.append(new_row)

# Convert the list of dictionaries to a DataFrame
final_dataframe = pd.DataFrame(data_list, columns=my_columns)

# Display the resulting DataFrame
print(final_dataframe)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PEAK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.

    Ticker   Price  Price-to-Earnings Ratio Number of Shares to Buy
0        A  149.76                35.827750                    None
1      AAL   14.40                20.281690                    None
2      AAP   75.03               150.060000                    None
3     AAPL  183.05                28.468119                    None
4     ABBV  160.75                47.842262                    None
..     ...     ...                      ...                     ...
500    YUM  137.62                24.400710                    None
501    ZBH  121.31                26.314533                    None
502   ZBRA  315.80                62.534650                    None
503   ZION   44.32                11.135678                    None
504    ZTS  169.04                32.570328                    None

[505 rows x 4 columns]


### **Removing Glamour Stocks**

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
# Remove each stocks that is no longer available in the Exchange (delisted etc.)
final_dataframe = final_dataframe.dropna(subset=['Price'])
# Remove Glamour
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

### **Calculating the Numver of Shares to BUY**

In [ ]:
# Enter portofolio size in dollars
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    print(f"Portofolio size in euros : {portfolio_size}")
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    
# Remove each stocks that is no longer available in the Exchange
final_dataframe = final_dataframe.dropna(subset=['Price'])

# Calculate the number of shared for an equal weighted protofolio

final_dataframe.reset_index(drop=True, inplace=True)
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

### **Building a Better (and More Realistic) Value Strategy**

Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
- Enterprise Value divided by Gross Profit (EV/GP)


In [13]:
# Define the new columns
rv_columns = [
    'Ticker', 'Price', 'Number of Shares to Buy', 
    'Price-to-Earnings Ratio', 'PE Percentile',
    'Price-to-Book Ratio', 'PB Percentile',
    'Price-to-Sales Ratio', 'PS Percentile',
    'EV/EBITDA', 'EV/EBITDA Percentile',
    'EV/GP', 'EV/GP Percentile', 'RV Score'
]

data_list = []

for ticker in stock_names['Ticker']:
    stock = yf.Ticker(ticker)
    info = stock.info

    row = {'Ticker': ticker, 'Number of Shares to Buy': 0}

    row['Price'] = info.get('currentPrice')
    row['Price-to-Earnings Ratio'] = info.get('trailingPE')
    row['Price-to-Book Ratio'] = info.get('priceToBook')
    row['Price-to-Sales Ratio'] = info.get('priceToSalesTrailing12Months')
    row['EV/EBITDA'] = (info.get('enterpriseValue', 0) / info.get('ebitda')) if info.get('ebitda') else None
    row['EV/GP'] = (info.get('enterpriseValue', 0) / info.get('grossProfits')) if info.get('grossProfits') else None
    
    data_list.append(row)

rv_dataframe = pd.DataFrame(data_list)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PEAK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.

In [19]:
# Remove each stocks that is no longer available in the Exchange (delisted etc.)
rv_dataframe = rv_dataframe.dropna(subset=['Price'])

Dealing with missing data is an important topic in data science.

There are two main approaches:

- Drop missing data from the data set (pandas' dropna method is useful here)
- Replace missing data with a new value (pandas' fillna method is useful here)

We replace missing data with the average of non-Nan Data 

In [20]:
#Remove Number of share to buy
del rv_dataframe['Number of Shares to Buy']
#Identify missing data
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
    
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

KeyError: 'Number of Shares to Buy'

In [21]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.696774
1      0.372043
2      0.991398
3      0.578495
4      0.877419
         ...   
500    0.470968
501    0.518280
502    0.929032
503    0.092473
504    0.638710
Name: PE Percentile, Length: 465, dtype: float64
0      0.709677
1      0.844086
2      0.215054
3      0.974194
4      0.972043
         ...   
500    0.844086
501    0.264516
502    0.619355
503    0.086022
504    0.916129
Name: PB Percentile, Length: 465, dtype: float64
0      0.825806
1      0.008602
2      0.045161
3      0.849462
4      0.748387
         ...   
500    0.767742
501    0.606452
502    0.647312
503    0.419355
504    0.907527
Name: PS Percentile, Length: 465, dtype: float64
0      0.911828
1      0.064516
2      0.772043
3      0.819355
4      0.369892
         ...   
500    0.754839
501    0.329032
502    0.901075
503    0.589247
504    0.847312
Name: EV/EBITDA Percentile, Length: 465, dtype: float64
0      0.501075
1      0.501075
2      0.501075
3      0.501075
4      0.501075
         ... 

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP,Price-to-Earnings Ratio Percentile,Price-to-Book Ratio Percentile,Price-to-Sales Ratio Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score,PE Percentile,PB Percentile,PS Percentile
0,A,149.76,35.827750,7.092252,6.516394,27.353023,0.0,0.696774,0.709677,0.825806,0.911828,0.501075,0.729032,0.696774,0.709677,0.825806
1,AAL,14.40,20.281690,11.288975,0.177692,6.820540,0.0,0.372043,0.844086,0.008602,0.064516,0.501075,0.358065,0.372043,0.844086,0.008602
2,AAP,75.03,150.060000,1.772083,0.396231,19.988622,0.0,0.991398,0.215054,0.045161,0.772043,0.501075,0.504946,0.991398,0.215054,0.045161
3,AAPL,183.05,28.468119,37.843708,7.355183,21.942186,0.0,0.578495,0.974194,0.849462,0.819355,0.501075,0.744516,0.578495,0.974194,0.849462
4,ABBV,160.75,47.842262,35.446530,5.217793,13.026239,0.0,0.877419,0.972043,0.748387,0.369892,0.501075,0.693763,0.877419,0.972043,0.748387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.62,24.400710,11.288975,5.514041,19.460789,0.0,0.470968,0.844086,0.767742,0.754839,0.501075,0.667742,0.470968,0.844086,0.767742
501,ZBH,121.31,26.314533,1.980733,3.348835,12.364644,0.0,0.518280,0.264516,0.606452,0.329032,0.501075,0.443871,0.518280,0.264516,0.606452
502,ZBRA,315.80,62.534650,5.122133,3.729501,26.456471,0.0,0.929032,0.619355,0.647312,0.901075,0.501075,0.719570,0.929032,0.619355,0.647312
503,ZION,44.32,11.135678,1.214313,2.242656,16.899094,0.0,0.092473,0.086022,0.419355,0.589247,0.501075,0.337634,0.092473,0.086022,0.419355


### **Calculating the RV Score**

We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

In [22]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Price-to-Book Ratio,Price-to-Sales Ratio,EV/EBITDA,EV/GP,Price-to-Earnings Ratio Percentile,Price-to-Book Ratio Percentile,Price-to-Sales Ratio Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score,PE Percentile,PB Percentile,PS Percentile
0,A,149.76,35.827750,7.092252,6.516394,27.353023,0.0,0.696774,0.709677,0.825806,0.911828,0.501075,0.729032,0.696774,0.709677,0.825806
1,AAL,14.40,20.281690,11.288975,0.177692,6.820540,0.0,0.372043,0.844086,0.008602,0.064516,0.501075,0.358065,0.372043,0.844086,0.008602
2,AAP,75.03,150.060000,1.772083,0.396231,19.988622,0.0,0.991398,0.215054,0.045161,0.772043,0.501075,0.504946,0.991398,0.215054,0.045161
3,AAPL,183.05,28.468119,37.843708,7.355183,21.942186,0.0,0.578495,0.974194,0.849462,0.819355,0.501075,0.744516,0.578495,0.974194,0.849462
4,ABBV,160.75,47.842262,35.446530,5.217793,13.026239,0.0,0.877419,0.972043,0.748387,0.369892,0.501075,0.693763,0.877419,0.972043,0.748387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.62,24.400710,11.288975,5.514041,19.460789,0.0,0.470968,0.844086,0.767742,0.754839,0.501075,0.667742,0.470968,0.844086,0.767742
501,ZBH,121.31,26.314533,1.980733,3.348835,12.364644,0.0,0.518280,0.264516,0.606452,0.329032,0.501075,0.443871,0.518280,0.264516,0.606452
502,ZBRA,315.80,62.534650,5.122133,3.729501,26.456471,0.0,0.929032,0.619355,0.647312,0.901075,0.501075,0.719570,0.929032,0.619355,0.647312
503,ZION,44.32,11.135678,1.214313,2.242656,16.899094,0.0,0.092473,0.086022,0.419355,0.589247,0.501075,0.337634,0.092473,0.086022,0.419355


In [ ]:
#Selecting the 50 Best Value Stocks
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating Number of Shares to Buy

# Enter portofolio size in dollars
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    print(f"Portofolio size in euros : {portfolio_size}")
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    
# Remove each stocks that is no longer available in the Exchange
rv_dataframe = rv_dataframe.dropna(subset=['Price'])

# Calculate the number of shared for an equal weighted protofolio

rv_dataframe.reset_index(drop=True, inplace=True)
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])


### **Formating the result in excel**

Using XlsxWriter

In [24]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

writer.close()